# مثال سیمپل ہوٹل اور فلائٹ بکنگ ایجنٹ

یہ حل آپ کو فلائٹ ٹکٹس اور ہوٹل بک کرنے میں مدد دے گا۔ منظرنامہ یہ ہے کہ لندن ہیثرو LHR سے نیو یارک JFK کے لیے 20 فروری 2024 کو سفر کرنا ہے اور 27 فروری 2025 کو واپس آنا ہے، اور صرف برٹش ایئر ویز کے ساتھ اکنامی کلاس میں سفر کرنا ہے۔ میں نیو یارک میں ہلٹن ہوٹل میں قیام کرنا چاہتا ہوں، براہ کرم فلائٹ اور ہوٹل دونوں کی قیمتیں فراہم کریں۔


# Azure AI Agent Service کو initialize کریں اور **.env** سے configuration معلومات حاصل کریں

### **.env** 

ایک .env فائل بنائیں

**.env** میں Azure AI Agent Service کا connection string، AOAI کے ذریعے استعمال ہونے والا ماڈل، اور متعلقہ Google API Search سروس کا API، ENDPOINT وغیرہ شامل ہوتے ہیں۔

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "آپ کے Azure AI Agent Service ماڈل ڈیپلائمنٹ کا نام"

[**NOTE**] آپ کو 100,000 Rate Limit (Tokens per minute)  اور 600 Rate Limit (Request per minute) والے ماڈل کی ضرورت ہوگی۔

  آپ Microsoft Foundry - Model and Endpoint میں ماڈل حاصل کر سکتے ہیں۔

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "آپ کے Azure AI Agent Service پروجیکٹ کا connection string"

  آپ اپنے پروجیکٹ کے overview میں AI ​​Foundry Portal Screen پر پروجیکٹ connection string حاصل کر سکتے ہیں۔

- **SERPAPI_SEARCH_API_KEY** = "آپ کا SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "آپ کا SERPAPI Search Endpoint"

Azure AI Agent Service کے Model Deployment Name اور Project Connection String حاصل کرنے کے لیے، آپ کو Azure AI Agent Service بنانا ہوگا۔ اس کے لیے یہ بہتر ہے کہ آپ [اس ٹیمپلیٹ](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) کو براہ راست استعمال کریں (***نوٹ:*** Azure AI Agent Service اس وقت محدود علاقے میں سیٹ ہے۔ بہتر ہوگا کہ آپ [اس لنک](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) کا حوالہ دے کر علاقہ سیٹ کریں)

Agent کو SERPAPI تک رسائی کی ضرورت ہے۔ رجسٹریشن کے لیے [اس لنک](https://serpapi.com/searches) کا استعمال کریں۔ رجسٹریشن کے بعد، آپ ایک منفرد API KEY اور ENDPOINT حاصل کر سکتے ہیں۔


# سیٹ اپ

اس نوٹ بک کو چلانے کے لیے، آپ کو یہ یقینی بنانا ہوگا کہ آپ نے ضروری لائبریریز `pip install -r requirements.txt` چلا کر انسٹال کر لی ہوں۔


In [ ]:
from semantic_kernel import __version__

__version__

آپ کے سیمانٹک کرنل کا ورژن کم از کم 1.27.2 ہونا چاہیے۔


براہ کرم اپنا .env فائل سیٹنگز اور وسائل لوڈ کریں یقینی بنائیں کہ آپ نے اپنی کیز اور سیٹنگز شامل کی ہیں اور ایک مقامی .env فائل بنائی ہے۔


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# لاگ ان کریں Azure میں

اب آپ کو Azure میں لاگ ان کرنے کی ضرورت ہے۔ ایک ٹرمینل کھولیں اور درج ذیل کمانڈ چلائیں:

```bash
az login
```

یہ کمانڈ آپ سے Azure کی اسناد داخل کرنے کو کہے گی، جس سے Azure AI Agent سروس صحیح طریقے سے کام کر سکے گی۔


# وضاحت:
یہ ایک متغیر ہے جو SERP (سرچ انجن نتائج کا صفحہ) API سروس تک رسائی کے لیے API کلید محفوظ کرتا ہے۔ API کلید ایک منفرد شناخت کنندہ ہے جو آپ کے اکاؤنٹ سے متعلق درخواستوں کی تصدیق کے لیے استعمال ہوتا ہے۔

مقصد: اس لائن کا مقصد API کلید کو ایک متغیر میں محفوظ کرنا ہے تاکہ اسے SERP API سروس کو درخواستوں کی تصدیق کے لیے استعمال کیا جا سکے۔ API کلید سروس تک رسائی اور تلاش کرنے کے لیے ضروری ہے۔
SERP API کلید کیسے حاصل کریں: SERP API کلید حاصل کرنے کے لیے، https://serpapi.com پر ان عمومی مراحل پر عمل کریں (مخصوص SERP API سروس کے مطابق مراحل مختلف ہو سکتے ہیں):

ایک SERP API سروس منتخب کریں: کئی SERP API سروسز دستیاب ہیں، جیسے SerpAPI، گوگل کسٹم سرچ JSON API، اور دیگر۔ اپنی ضروریات کے مطابق بہترین سروس کا انتخاب کریں۔

اکاؤنٹ کے لیے سائن اپ کریں: منتخب کردہ SERP API سروس کی ویب سائٹ پر جائیں اور اکاؤنٹ کے لیے سائن اپ کریں۔ آپ کو کچھ بنیادی معلومات فراہم کرنے اور اپنا ای میل ایڈریس تصدیق کرنے کی ضرورت ہو سکتی ہے۔

API کلید بنائیں: سائن اپ کرنے کے بعد، اپنے اکاؤنٹ میں لاگ ان کریں اور API سیکشن یا ڈیش بورڈ پر جائیں۔ نئی API کلید بنانے یا پیدا کرنے کا آپشن تلاش کریں۔
API کلید کو اپنی .env فائل میں کاپی کریں۔


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# وضاحت:
BASE_URL: یہ ایک متغیر ہے جو SERP API اینڈپوائنٹ کے لیے بنیادی URL ذخیرہ کرتا ہے۔ متغیر کا نام BASE_URL ایک روایت ہے جو یہ ظاہر کرتی ہے کہ یہ URL API درخواستیں کرنے کے لیے ابتدائی نقطہ آغاز ہے۔
'https://serpapi.com/search':

یہ وہ اصل URL اسٹرنگ ہے جو BASE_URL متغیر کو سونپی گئی ہے۔ یہ SERP API کا استعمال کرتے ہوئے تلاش کے سوالات انجام دینے کے اینڈپوائنٹ کی نمائندگی کرتا ہے۔

# مقصد:
اس لائن کا مقصد ایک مستقل تعریف کرنا ہے جو SERP API کے لیے بنیادی URL رکھتا ہے۔ یہ URL API درخواستوں کی تشکیل کے لیے ابتدائی نقطہ آغاز کے طور پر استعمال ہوگا تاکہ تلاش کے عملیات انجام دیے جا سکیں۔

# استعمال:
بنیادی URL کو متغیر میں تعریف کر کے، آپ اسے اپنے کوڈ میں آسانی سے دوبارہ استعمال کر سکتے ہیں جب بھی آپ کو SERP API کو درخواستیں بھیجنی ہوں۔ اس سے آپ کا کوڈ زیادہ قابلِ نگہداشت بنتا ہے اور متعدد جگہوں پر URL ہارڈ کوڈ کرنے کی وجہ سے غلطیوں کا خطرہ کم ہو جاتا ہے۔ موجودہ مثال https://serpapi.com/search?engine=bing ہے جو Bing سرچ API استعمال کر رہی ہے۔ مختلف API https://Serpapi.com پر منتخب کی جا سکتی ہے۔


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# وضاحت:

یہاں آپ کے پلگ ان کا کوڈ واقع ہے۔

کلاس کی تعریف: `class BookingPlugin`: ایک کلاس تعریف کرتا ہے جس کا نام BookingPlugin ہے جو ہوٹل اور پروازوں کی بکنگ کے طریقے رکھتی ہے۔

ہوٹل بکنگ کا طریقہ:

- `@kernel_function(description="booking hotel")`: ایک ڈیکوریٹر ہے جو فنکشن کو ہوٹلوں کی بکنگ کے لیے کرنل فنکشن کے طور پر بیان کرتا ہے۔
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: ایک ایسا طریقہ تعریف کرتا ہے جو تفویض کردہ پیرا میٹرز اور واپسی کی قسم کے ساتھ ہوٹلز کی بکنگ کے لیے ہے۔

یہ طریقہ ہوٹل بکنگ کی درخواست کے لیے پیرامیٹرز کا ایک ڈکشنری بناتا ہے اور SERP API کو GET درخواست بھیجتا ہے۔ یہ رد عمل کی حالت کو چیک کرتا ہے اور اگر کامیاب ہو تو ہوٹل کی خصوصیات واپس کرتا ہے، ورنہ اگر درخواست ناکام ہو جائے تو None واپس کرتا ہے۔

پرواز کی بکنگ کا طریقہ:

- `@kernel_function(description="booking flight")`: ایک ڈیکوریٹر ہے جو فنکشن کو پروازوں کی بکنگ کے لیے کرنل فنکشن کے طور پر بیان کرتا ہے۔
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: ایک ایسا طریقہ تعریف کرتا ہے جو تفویض کردہ پیرا میٹرز اور واپسی کی قسم کے ساتھ پروازوں کی بکنگ کے لیے ہے۔

یہ طریقہ روانگی اور واپسی کی پروازوں کی درخواست کے پیرامیٹرز کے ڈکشنریاں بناتا ہے اور SERP API کو GET درخواستیں بھیجتا ہے۔ یہ ردعمل کی حالت کو چیک کرتا ہے اور اگر کامیاب ہو تو نتیجہ کے سٹرنگ میں پرواز کی معلومات شامل کرتا ہے، ورنہ اگر درخواست ناکام ہو تو ایک غلطی کا پیغام پرنٹ کرتا ہے۔ طریقہ ایسے نتیجہ کی سٹرنگ واپس کرتا ہے جس میں پرواز کی معلومات شامل ہوتی ہے۔


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# وضاحت:
درآمد کے بیانات: Azure کی اسناد، AI ایجنٹ، چیٹ پیغام کے مواد، مصنف کے کردار، اور کرنل فنکشن ڈیکوریٹر کے لئے ضروری ماڈیول درآمد کریں۔

غیر متزلزل سیاق و سباق منیجر: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): یہ Azure کی اسناد کو ہینڈل کرنے اور AI ایجنٹ کلائنٹ بنانے کے لیے ایک غیر متزلزل سیاق و سباق منیجر ترتیب دیتا ہے۔

ایجنٹ کا نام اور ہدایات:
- `AGENT_NAME = "BookingAgent"`: ایجنٹ کا نام متعین کرتا ہے۔
- `AGENT_INSTRUCTIONS = """..."""`: ایجنٹ کو بکنگ کی درخواستوں کو کیسے سنبھالنا ہے اس کے بارے میں تفصیلی ہدایات فراہم کرتا ہے۔

ایجنٹ کی تعریف بنائیں: `agent_definition = await client.agents.create_agent(...)`: مخصوص ماڈل، نام، اور ہدایات کے ساتھ ایجنٹ کی تعریف بناتا ہے۔

AzureAI ایجنٹ بنائیں: `agent = AzureAIAgent(...)`: کلائنٹ، ایجنٹ کی تعریف، اور متعین پلگ ان کا استعمال کرتے ہوئے AzureAI ایجنٹ بناتا ہے۔

تھریڈ بنائیں: `thread: AzureAIAgentThread | None = None`: ایجنٹ کے لیے ایک تھریڈ بنائیں۔ پہلے تھریڈ بنانا لازمی نہیں - اگر `None` کی قدر دی جائے تو، پہلی کال پر ایک نئی تھریڈ بنائی جائے گی اور جواب کے حصے کے طور پر واپس کی جائے گی۔

صارف کے ان پٹ: `user_inputs = ["..."]`: ایجنٹ کے عمل کے لیے صارف کے ان پٹ کی فہرست متعین کرتا ہے۔

آخر میں، resources کی صفائی کے لیے تھریڈ اور ایجنٹ کو حذف کریں۔


# توثیق

`DefaultAzureCredential` کلاس Azure SDK for Python کا حصہ ہے۔ یہ Azure خدمات کے ساتھ تصدیق کرنے کا ایک ڈیفالٹ طریقہ فراہم کرتا ہے۔ یہ مخصوص ترتیب میں متعدد طریقوں کو آزما کر تصدیق کرنے کی کوشش کرتا ہے، جیسے ماحولیاتی متغیرات، منیجڈ شناخت، اور Azure CLI اسناد۔

غیر متزامن آپریشنز: aio ماڈیول اس بات کی علامت ہے کہ DefaultAzureCredential کلاس غیر متزامن آپریشنز کی حمایت کرتی ہے۔ اس کا مطلب ہے کہ آپ اسے asyncio کے ساتھ استعمال کر کے غیر بلاک کرنے والی تصدیقی درخواستیں انجام دے سکتے ہیں۔


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**دستاویزی اعلان**:
اس دستاویز کا ترجمہ اے آئی ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے کیا گیا ہے۔ جب کہ ہم درستگی کے لیے کوشاں ہیں، براہ کرم نوٹ کریں کہ خودکار ترجموں میں غلطیاں یا عدم درستیاں ہو سکتی ہیں۔ اصل دستاویز اپنی مادری زبان میں معتبر ماخذ سمجھی جائے۔ اہم معلومات کے لیے پیشہ ورانہ انسانی ترجمہ تجویز کیا جاتا ہے۔ ہم اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تعبیر کے ذمہ دار نہیں ہیں۔
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
